In [6]:
# !pip install optuna

In [7]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [9]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [14]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-08-16 21:56:48,670] A new study created in memory with name: no-name-24b699c4-9096-4fbd-a9ae-04297086bc85
[I 2025-08-16 21:56:48,887] Trial 0 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 60, 'max_depth': 4}. Best is trial 0 with value: 0.7560521415270017.
[I 2025-08-16 21:56:49,175] Trial 1 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 82, 'max_depth': 6}. Best is trial 0 with value: 0.7560521415270017.
[I 2025-08-16 21:56:49,859] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 173, 'max_depth': 13}. Best is trial 2 with value: 0.7709497206703911.
[I 2025-08-16 21:56:50,119] Trial 3 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 67, 'max_depth': 17}. Best is trial 3 with value: 0.7783985102420857.
[I 2025-08-16 21:56:50,385] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 71, 'max_depth': 16}. Best is trial 3 with value: 0.7783985102

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 70, 'max_depth': 18}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [17]:
#Sampler optuna

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [19]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-08-16 21:57:50,966] A new study created in memory with name: no-name-777f0317-f8bf-4d48-b7d1-1e22ed1688fc
[I 2025-08-16 21:57:52,497] Trial 0 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 142, 'max_depth': 3}. Best is trial 0 with value: 0.7579143389199254.
[I 2025-08-16 21:57:53,213] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 58, 'max_depth': 13}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-08-16 21:57:54,727] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 96, 'max_depth': 20}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-08-16 21:57:56,093] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 62, 'max_depth': 7}. Best is trial 3 with value: 0.7765363128491621.
[I 2025-08-16 21:57:58,388] Trial 4 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 86, 'max_depth': 4}. Best is trial 3 with value: 0.77653631284

In [20]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 54, 'max_depth': 7}


In [21]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [22]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [23]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-08-16 21:58:56,031] A new study created in memory with name: no-name-db9c806c-1954-4d9f-997c-31f29fb1871a
[I 2025-08-16 21:58:56,361] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-16 21:58:56,951] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-16 21:58:57,148] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-16 21:58:57,561] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-16 21:58:57,932] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [24]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [25]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [26]:
#optuna visualisation

In [27]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [28]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [29]:
# 3. Slice Plot
plot_slice(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [30]:
# 4. Contour Plot
plot_contour(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [31]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [32]:
#Optimaising ML multiple Models

In [33]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [34]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [35]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-16 22:00:45,776] A new study created in memory with name: no-name-c665052c-f76b-4b60-b574-0bb45b637d91
[I 2025-08-16 22:00:47,781] Trial 0 finished with value: 0.7690875232774674 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 96, 'learning_rate': 0.02051608719951715, 'max_depth': 14, 'min_samples_split': 9, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-16 22:00:47,841] Trial 1 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 1.9294669536846905, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.7858472998137801.
[I 2025-08-16 22:01:01,062] Trial 2 finished with value: 0.7374301675977654 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 256, 'learning_rate': 0.058740592869685956, 'max_depth': 16, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.7858472998137801.
[I 2025-08-16 22:01:26,148] Trial 3 finished with value: 0.661

In [36]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11889704493610485, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [37]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.769088,2025-08-16 22:00:45.782094,2025-08-16 22:00:47.781680,0 days 00:00:01.999586,NaN,NaN,GradientBoosting,NaN,NaN,0.020516,14.0,10.0,9.0,96.0,COMPLETE
1,1,0.785847,2025-08-16 22:00:47.781680,2025-08-16 22:00:47.841584,0 days 00:00:00.059904,1.929467,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.737430,2025-08-16 22:00:47.844172,2025-08-16 22:01:01.062618,0 days 00:00:13.218446,NaN,NaN,GradientBoosting,NaN,NaN,0.058741,16.0,8.0,5.0,256.0,COMPLETE
3,3,0.661080,2025-08-16 22:01:01.062618,2025-08-16 22:01:26.148774,0 days 00:00:25.086156,NaN,NaN,GradientBoosting,NaN,NaN,0.067525,20.0,1.0,4.0,202.0,COMPLETE
4,4,0.757914,2025-08-16 22:01:26.153423,2025-08-16 22:01:28.109371,0 days 00:00:01.955948,NaN,True,RandomForest,NaN,NaN,NaN,11.0,6.0,9.0,96.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.698324,2025-08-16 22:02:05.455668,2025-08-16 22:02:05.544078,0 days 00:00:00.088410,6.562760,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.750466,2025-08-16 22:02:05.544078,2025-08-16 22:02:05.669248,0 days 00:00:00.125170,0.129737,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.765363,2025-08-16 22:02:05.669248,2025-08-16 22:02:09.487725,0 days 00:00:03.818477,NaN,False,RandomForest,NaN,NaN,NaN,12.0,3.0,5.0,262.0,COMPLETE
98,98,0.746741,2025-08-16 22:02:09.487725,2025-08-16 22:02:19.520295,0 days 00:00:10.032570,NaN,NaN,GradientBoosting,NaN,NaN,0.033066,20.0,9.0,3.0,223.0,COMPLETE


In [38]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 78
GradientBoosting    12
RandomForest        10
Name: count, dtype: int64

In [39]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.739137
RandomForest        0.766853
SVM                 0.776035
Name: value, dtype: float64

In [43]:
pip install optuna[visualization]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import plotly
print(plotly.__version__)

6.3.0


In [50]:
# 1. Optimization History
from optuna.visualization import plot_optimization_history
plot_optimization_history(study).show()


ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [51]:
# 3. Slice Plot
plot_slice(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [52]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [55]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-08-16 22:07:47,863] A new study created in memory with name: no-name-541da5c9-06f0-4902-993c-10aa7f1b7e1c


[0]	train-mlogloss:0.81681	eval-mlogloss:0.80563
[1]	train-mlogloss:0.71075	eval-mlogloss:0.69364
[2]	train-mlogloss:0.57124	eval-mlogloss:0.54449
[3]	train-mlogloss:0.45411	eval-mlogloss:0.41671
[4]	train-mlogloss:0.37084	eval-mlogloss:0.33212
[5]	train-mlogloss:0.30803	eval-mlogloss:0.26361
[6]	train-mlogloss:0.26238	eval-mlogloss:0.20851
[7]	train-mlogloss:0.23028	eval-mlogloss:0.17089
[8]	train-mlogloss:0.20750	eval-mlogloss:0.14298
[9]	train-mlogloss:0.19483	eval-mlogloss:0.13404
[10]	train-mlogloss:0.19130	eval-mlogloss:0.13363
[11]	train-mlogloss:0.18484	eval-mlogloss:0.12604
[12]	train-mlogloss:0.18347	eval-mlogloss:0.12587
[13]	train-mlogloss:0.17866	eval-mlogloss:0.11895
[14]	train-mlogloss:0.17848	eval-mlogloss:0.11850
[15]	train-mlogloss:0.17820	eval-mlogloss:0.11799
[16]	train-mlogloss:0.17720	eval-mlogloss:0.11728
[17]	train-mlogloss:0.17705	eval-mlogloss:0.11734
[18]	train-mlogloss:0.17715	eval-mlogloss:0.11701
[19]	train-mlogloss:0.17027	eval-mlogloss:0.10959
[20]	train

[I 2025-08-16 22:07:49,959] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.00014405154340233053, 'alpha': 1.0028087182541891e-05, 'eta': 0.24399632761008344, 'gamma': 3.146022815330369e-05, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.5976578373605221, 'colsample_bytree': 0.6061634604717767}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.98739	eval-mlogloss:0.98306
[1]	train-mlogloss:0.89280	eval-mlogloss:0.88517
[2]	train-mlogloss:0.80992	eval-mlogloss:0.79829
[3]	train-mlogloss:0.73773	eval-mlogloss:0.72202
[4]	train-mlogloss:0.67499	eval-mlogloss:0.65742
[5]	train-mlogloss:0.61922	eval-mlogloss:0.59873
[6]	train-mlogloss:0.57064	eval-mlogloss:0.54661
[7]	train-mlogloss:0.52641	eval-mlogloss:0.50025
[8]	train-mlogloss:0.48771	eval-mlogloss:0.45926
[9]	train-mlogloss:0.45182	eval-mlogloss:0.42107
[10]	train-mlogloss:0.41901	eval-mlogloss:0.39035
[11]	train-mlogloss:0.38951	eval-mlogloss:0.35923
[12]	train-mlogloss:0.36330	eval-mlogloss:0.33190
[13]	train-mlogloss:0.33800	eval-mlogloss:0.30601
[14]	train-mlogloss:0.31605	eval-mlogloss:0.28388
[15]	train-mlogloss:0.29632	eval-mlogloss:0.26322
[16]	train-mlogloss:0.27849	eval-mlogloss:0.24457
[17]	train-mlogloss:0.26220	eval-mlogloss:0.22691
[18]	train-mlogloss:0.24727	eval-mlogloss:0.21145
[19]	train-mlogloss:0.23307	eval-mlogloss:0.19776
[20]	train

[I 2025-08-16 22:07:50,769] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.5735843410244273, 'alpha': 2.0494419209983586e-06, 'eta': 0.08777659961553262, 'gamma': 0.009276442907966639, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.9718450880936284, 'colsample_bytree': 0.9964878436449474}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.06862	eval-mlogloss:1.06771
[1]	train-mlogloss:1.04039	eval-mlogloss:1.03764
[2]	train-mlogloss:1.01185	eval-mlogloss:1.00783
[3]	train-mlogloss:0.98483	eval-mlogloss:0.98026


[I 2025-08-16 22:07:59,202] Trial 48 pruned. Trial was pruned at iteration 4.


[0]	train-mlogloss:0.94305	eval-mlogloss:0.93614
[1]	train-mlogloss:0.86634	eval-mlogloss:0.85893


[I 2025-08-16 22:07:59,308] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 0.00014405154340233053, 'alpha': 1.0028087182541891e-05, 'eta': 0.24399632761008344, 'gamma': 3.146022815330369e-05, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.5976578373605221, 'colsample_bytree': 0.6061634604717767}
Best accuracy: 1.0


In [56]:
# ! pip install optuna-integration[xgboost]

In [57]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.